In [5]:
%%writefile client.py


import cv2
import os
import time
import threading
time.sleep(1)

url =  "http://127.0.0.1:3000/video_feed"

# Initialize VideoCapture object
cap = cv2.VideoCapture(url)

# Get the default frame size
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 3.6, (frame_width, frame_height))

# Function to continuously read frames and write them to the output video
def read_frames():
    frame_count = 0
    start_time = time.time()
    while frame_count < 100:
        ret, frame = cap.read()
        if ret:
            frame_count += 1
            out.write(frame)
        else:
            print("Error reading frame")
            break
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Time taken to get 100 frames:", elapsed_time, "seconds")

# Start reading frames in a separate thread
frame_thread = threading.Thread(target=read_frames)
frame_thread.start()

# Wait for the frame thread to finish
frame_thread.join()

# Release everything when done
cap.release()
out.release()
cv2.destroyAllWindows()


Overwriting client.py


In [6]:

%%writefile client1.py

import cv2
import os
import time
import threading
time.sleep(1)

url =  "http://127.0.0.1:3000/video_feed"

# Initialize VideoCapture object
cap = cv2.VideoCapture(url)

# Get the default frame size
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output1.avi', fourcc, 3.6, (frame_width, frame_height))

# Function to continuously read frames and write them to the output video
def read_frames():
    frame_count = 0
    start_time = time.time()
    while frame_count < 100:
        ret, frame = cap.read()
        if ret:
            frame_count += 1
            out.write(frame)
        else:
            print("Error reading frame")
            break
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Time taken to get 100 frames:", elapsed_time, "seconds")

# Start reading frames in a separate thread
frame_thread = threading.Thread(target=read_frames)
frame_thread.start()

# Wait for the frame thread to finish
frame_thread.join()

# Release everything when done
cap.release()
out.release()
cv2.destroyAllWindows()

Overwriting client1.py


In [3]:
!python client.py & python client1.py

Time taken to get 100 frames: 25.519768714904785 seconds
Time taken to get 100 frames: 25.84660029411316 seconds


In [3]:

%%writefile all.py
import torch.multiprocessing as mp
import time
import subprocess
import torch
print('starting.....')
def run_script(script):
    """Function to run a script."""
    subprocess.run(["python", script])

if __name__ == "__main__":
    start_time = time.time()
    
    # List of scripts to run
    scripts = ["client.py",'client1.py']
    
    # Number of processes to use
    num_processes = len(scripts)
    
    # Create a list to hold process objects
    processes = []
    
    # Start processes
    for script in scripts:
        p = mp.Process(target=run_script, args=(script,))
        p.start()
        processes.append(p)
    
    # Join processes
    for p in processes:
        p.join()
    
    # Record end time
    end_time = time.time()
    
    # Calculate and print total processing time
    total_time = end_time - start_time
    print(f"Total processing time: {total_time} seconds")


Overwriting all.py


In [4]:
!python all.py

Time taken to get 100 frames: 29.449400663375854 seconds
starting.....
Time taken to get 100 frames: 36.136369466781616 seconds
starting.....
starting.....
Total processing time: 40.95079278945923 seconds


In [1]:
import multiprocessing
import time
import subprocess

def run_script(script):
    """Function to run a script."""
    subprocess.run(["python", script])

if __name__ == "__main__":
    start_time = time.time()
    
    # List of scripts to run
    scripts = ["client.py", "client1.py"]
    
    # Number of processes to use
    num_processes = len(scripts)
    
    # Create a list to hold process objects
    processes = []
    
    # Start processes
    for script in scripts:
        p = multiprocessing.Process(target=run_script, args=(script,))
        p.start()
        processes.append(p)
    
    # Join processes
    for p in processes:
        p.join()
    
    # Record end time
    end_time = time.time()
    
    # Calculate and print total processing time
    total_time = end_time - start_time
    print(f"Total processing time: {total_time} seconds")


Total processing time: 0.24500203132629395 seconds


In [ ]:
from fastapi import FastAPI
from starlette.responses import StreamingResponse
import cv2
import time
import uvicorn
import numpy as np
import datetime
import asyncio
from queue import Queue
import nest_asyncio
nest_asyncio.apply()
app = FastAPI()

# Shared variable to store the latest frame
latest_frame = None

async def generate_frames():
    global latest_frame
    c = 0
    # Variable for frame interval (in seconds) to achieve desired FPS
    frame_interval = 1 / 4
    start_time = time.time()
    num_frames = 0
    url = "https://video.queendahyun.com/video_feed"  # Update the URL if needed
    cap = cv2.VideoCapture(url)
    while True:
        print(f'total {c}')
        # Record start time of processing a frame
        frame_start_time = time.time()
        ret, frame = cap.read()

        if True:
            img_final = cv2.resize(frame, (1280, 720))  # 480p resolution is 854x480
            # Convert the frame to JPEG format for streaming
            _, buffer = cv2.imencode('.jpg', img_final)
            c += 1
            num_frames += 1

            latest_frame = buffer.tobytes()  # Update the latest frame

        # Calculate elapsed time
        elapsed_time = time.time() - start_time

        # If one second has elapsed, calculate and print FPS
        if elapsed_time >= 1:
            fps = num_frames / elapsed_time
            # print("FPS:", fps)

        frame_time = time.time() - frame_start_time
        # Calculate the delay required to achieve desired FPS
        delay = frame_interval - frame_time

        try:
            # Introduce the delay
            await asyncio.sleep(delay)
        except asyncio.CancelledError:
            break

@app.get("/video_feed")
async def video_feed():
    async def stream_frame():
        global latest_frame
        while True:
            start_time = time.time()  # Record start time of sending frames
            for _ in range(1):  # Send 4 frames per second
                if latest_frame is not None:  # Check if there's a latest frame available
                    yield (
                        b'--frame\r\n'
                        b'Content-Type: image/jpeg\r\n\r\n' + latest_frame + b'\r\n'
                    )
                else:
                    await asyncio.sleep(0.1)  # If no latest frame available, wait for a short time

            # Calculate elapsed time and wait if necessary to maintain FPS rate
            elapsed_time = time.time() - start_time
            if elapsed_time < 1 / 4:
                await asyncio.sleep(1 / 4 - elapsed_time)

    return StreamingResponse(stream_frame(), media_type="multipart/x-mixed-replace; boundary=frame")

async def main():
    # Start generating frames asynchronously
    asyncio.create_task(generate_frames())

if __name__ == "__main__":
    asyncio.run(main())
    uvicorn.run(app, host="127.0.0.1", port=8000)


INFO:     Started server process [5396]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


total 0
total 1
total 2
total 3
total 4
total 5
total 6
total 7
total 8
total 9
total 10
total 11
total 12
total 13
total 14
total 15
total 16
total 17
total 18
total 19
total 20
total 21
total 22
total 23
total 24
total 25
total 26
total 27
total 28
total 29
total 30
total 31
total 32
total 33
total 34
total 35
total 36
total 37
total 38
total 39
total 40
total 41
total 42
total 43
total 44
total 45
total 46
total 47
total 48
total 49
total 50
total 51
total 52
total 53
total 54
total 55
total 56
total 57
total 58
total 59
total 60
total 61
total 62
total 63
total 64
total 65
total 66
total 67
total 68
total 69
total 70
total 71
total 72
total 73
total 74
total 75
total 76
total 77
total 78
total 79
total 80
total 81
total 82
total 83
total 84
total 85
total 86
total 87
total 88
total 89
total 90
total 91
total 92
total 93
total 94
total 95
total 96
total 97
total 98
total 99
total 100
total 101
total 102
total 103
total 104
total 105
total 106
total 107
total 108
total 109
total 110


In [1]:
TOTAL_FLOPS = 1.3 GHz * 768 cores * 2 FLOPS

SyntaxError: invalid syntax (3618676740.py, line 1)